---
title: Inline Interactivity in Jupyter using MyST
subtitle: Using inline widgets in JupyterLab markdown cells
author:
  - name: Rowan Cockett
    affiliations: Executable Books; Curvenote
    orcid: 0000-0002-7859-8394
    email: rowan@curvenote.com
  - name: Angus Hollands
    affiliations: Executable Books
  - name: Steve Purves
    affiliations: Executable Books; Curvenote
date: 2023/02/20
---

The [JupyterLab MyST extension](https://github.com/executablebooks/jupyterlab-myst) allows you to have MyST renderer in your markdown cells that includes interactivity and inline-evaluation.

This is done through the `` {eval}`1+1` `` role, which results in {eval}`1+1`. The extension can also access variables in the kernel.

:::{warning} Syntax is Subject to Change
:class: dropdown
The current syntax is based on a myst role, which may change in the future -- for example to be closer to an inline expression `${}`. This will go through a MyST enhancement proposal, and the `{eval}` role will likely still be supported.
:::

In [3]:
import numpy as np
array = np.arange(4)

Let's consider the following array: {eval}`array`.

We can compute the total: {eval}`array.sum()` and the maximum value is {eval}`array.max()`.

This is done through using MyST syntax: `` {eval}`array.sum()` `` and `` {eval}`array.max()` ``, respectively.

In [80]:
import ipywidgets as widgets

cookiesSlider = widgets.IntSlider(min=0, max=30, step=1, value=10, description="Cookies: ")
cookiesText = widgets.BoundedIntText(
    value=10,
    min=0,
    max=30,
    step=1,
)
widgetLink = widgets.jslink((cookiesSlider, 'value'), (cookiesText, 'value'))

caloriesPerCookie = 50
dailyCalories = 2100.

calories = widgets.Label(value=f'{cookiesSlider.value * caloriesPerCookie}')
def fc(n):
    calories.value = f'{n["owner"].value * caloriesPerCookie}'
cookiesSlider.observe(fc)


percent = widgets.Label(value=f'{cookiesSlider.value * caloriesPerCookie / dailyCalories * 100}')
def fp(n):
    percent.value = f'{n["owner"].value * caloriesPerCookie / dailyCalories * 100}'
cookiesSlider.observe(fp)

:::{important} Calculate Consuming Cookie Calories 🍪 🧮
When you eat {eval}`cookiesText` cookies, you consume {eval}`calories` calories. \
That's {eval}`percent`% of your recommended daily calories.
:::

You can also edit this through a slider if you want: {eval}`cookiesSlider`

:::{note} Improvements
:class: dropdown
There is a lot to improve both for formatting (e.g. format the numbering) and for the verbosity of the above code for linking and calculating the widgets.
These are probably going to be a combination of a MyST library for working with simple inline widgets and showing numbers, as well as improvements and tweaks to the inline renderers for the mime bundles.

But you can already do things! 🚀
:::

In [84]:
from IPython.display import Image, HTML

i = Image(url='https://source.unsplash.com/random/400x50?sunset')

This should be an inline image of a sunset: {eval}`i`

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Predict the stock market, with surprising accuracy:
data = np.cumsum(np.random.rand(1000)-0.5)
data = data - np.mean(data)

stocks = plt.figure(figsize=(10,2))
ax1 = stocks.add_subplot(111)
ax1.plot(data, 'b-')
ax1.axhline(c='grey', alpha=0.5);
plt.axis('off')
plt.tight_layout();
plt.close();

annotated = plt.figure(figsize=(10,2))
ax1 = annotated.add_subplot(111)
plt.axvspan(450, 560, color='red', alpha=0.5)
plt.axvspan(680, 740, color='green', alpha=0.5)
ax1.plot(data, 'b-')
ax1.axhline(c='grey', alpha=0.5);
plt.axis('off')
plt.tight_layout();
plt.close();


If we look at the following stock price for [Apple](https://en.wikipedia.org/wiki/Apple_Inc.) ({eval}`stocks`) we can see that in 2003, they started selling [computers](https://en.wikipedia.org/wiki/MacBook_Pro), in the red region and their stock went crazy once the investment paid off ({eval}`annotated`).[^note]

[^note]: Both the graphs are fake, but pretty cool right!  

In [86]:
text_hover = HTML('<span onmouseover="this.innerText=\'💚\'" onmouseout="this.innerText=\'🎉\'">❤️</span>');

Try hovering over this element[^trust]: {eval}`text_hover`

[^trust]: You may need to set your notebook to trusted to do this!